In [1]:
import pandas as pd
import random
import time, json
import datetime
data = pd.read_excel('./用户.xlsx')

In [2]:
def strTimeProp(start, end, prop, frmt):
    stime = time.mktime(time.strptime(start, frmt))
    etime = time.mktime(time.strptime(end, frmt))
    ptime = stime + prop * (etime - stime)
    return int(ptime)

In [3]:
def randomDate(start, end, frmt='%Y-%m-%d %H:%M:%S'):
    return time.strftime(frmt, time.localtime(strTimeProp(start, end, random.random(), frmt)))

In [4]:
def randomDateList(start, end, n, frmt='%Y-%m-%d %H:%M:%S'):
    return [randomDate(start, end, frmt) for _ in range(n)]

## 创建用户分数

In [30]:
class Create: 
    def __init__(self, itemid, max_security_score = 74):
        self.itemid = itemid
        self.max_security_score = max_security_score
        
        self.data = pd.DataFrame()
        self.data['datetime'] = pd.date_range(start = '2021-07-01', end = '2021-09-14', freq = '1D')
        self.data['mingan_score'] = self.data.apply(lambda x: random.randint(10,self.max_security_score), axis = 1)
        self.data['lizhi_score'] = self.data.apply(lambda x: random.randint(10,self.max_security_score), axis = 1)
        self.data['siren_score'] = self.data.apply(lambda x: random.randint(10,self.max_security_score), axis = 1)
        self.data['guanli_score'] = self.data.apply(lambda x: random.randint(10,self.max_security_score), axis = 1)
        self.data['beicai_score'] = self.data.apply(lambda x: random.randint(10,self.max_security_score), axis = 1)
        
    def change_score(self, col, start_anomaly_timelist, end_anomaly_timelist):
        # 异常分数设定
        for i in range(len(start_anomaly_timelist)):
            self.data[col] = self.data.apply(lambda x: random.randint(75,90) if (str(x['datetime']) > start_anomaly_timelist[i])& (str(x['datetime']) < end_anomaly_timelist[i]) else x[col], axis = 1)
    
    def save_json(self):
        return self.data.to_json('./user_json/{}.json'.format(self.itemid),orient = 'records')

In [31]:
start_date = ['2021-09-01', '2021-09-04', '2021-09-06']
end_date = ['2021-09-03', '2021-09-05', '2021-09-08']
create = Create('JLM0364')
create.change_score('mingan_score', start_date, end_date)
create.change_score('siren_score', start_date, end_date)
create.save_json()

## 为高于75分的数据创建事件

In [6]:
events = {}
events['敏感数据泄露'] = '测试页面泄露在外网，管理后台地址泄漏，QQ群敏感文件泄露，百度网盘敏感信息泄露，github源码泄露，运维监控系统弱口令泄露，Http认证信息泄露'.split('，')
events['离职倾向用户风险'] = '长期停留在招聘网站，打开外部网页频率增加，文件上传百度网盘频率增加，某就业网站访问频率增加，网页查询消极信息频率增高'.split('，')
events['私人文件盗取'] = 'github源码频繁下载'.split('，')
events['管理员账号泄露'] = '账号异地登录，账号被多次输错后登录，账号不在工作时间内登录'.split('，')
events['被裁员用户风险'] = '被多次线上约谈'.split('，')

In [47]:
class Change:
    def __init__(self, itemid):
        self. itemid = itemid
        with open('./user_json/{}.json'.format(self.itemid),'r',encoding='utf8')as fp:
            self.json_data = json.load(fp)
    def change(self, number, col_score, col_event, info):
        for line in self.json_data:
            line['timestamp'] = str(pd.to_datetime(datetime.datetime.fromtimestamp(int((line['datetime']-8*60*60*1000)/1000))))
            if line[col_score] >= 75:
                start = line['timestamp']
                end = str(pd.to_datetime(datetime.datetime.fromtimestamp(int((line['datetime']-8*60*60*1000)/1000))+datetime.timedelta(days=1)))
                lenth = number
                randomdata = randomDateList(start, end, lenth)
                datedict = {}
                for i in randomdata:
                    datedict[i] = random.sample(events[info], 1)[0]
                line[col_event] = json.dumps(datedict, ensure_ascii = False)
            else:
                line[col_event] = ''
        
            
    def multichange(self):
        self.change(2, 'mingan_score', 'mingan_event', '敏感数据泄露')
        self.change(2, 'lizhi_score', 'lizhi_event', '离职倾向用户风险')
        self.change(3, 'siren_score', 'siren_event', '私人文件盗取')
        self.change(2, 'guanli_score', 'guanli_event', '管理员账号泄露')
        self.change(1, 'beicai_score', 'beicai_event', '被裁员用户风险')

In [49]:
change = Change('JLM0364')
change.multichange()

## 上传elasticsearch数据

In [52]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [53]:
es = Elasticsearch('192.168.50.98:19201')

In [64]:
class Upload:
    def __init__(self, json_data):
        self.json_data = json_data
        
    def Actions(self,col_score, col_event, name):
        ACTIONS = []
        for line in self.json_data:
            action = {
                "_index": 'dc_emp_data',
                "_type": '_doc',
                "_source": {
                    "user": 'JLM0364',
                    "score": line[col_score],
                    "scenario": name,
                    "events": line[col_event],
                    "datetime": line['timestamp']}
            }
            ACTIONS.append(action)
        return ACTIONS
    
    def upload(self):
        Actions1 = Actions(self.json_data, 'mingan_score', 'mingan_event', '敏感数据泄露')
        Actions2 = Actions(self.json_data, 'lizhi_score', 'lizhi_event', '离职倾向用户风险')
        Actions3 = Actions(self.json_data, 'siren_score', 'siren_event', '私人文件盗取')
        Actions4 = Actions(self.json_data, 'guanli_score', 'guanli_event', '管理员账号泄露')
        Actions5 = Actions(self.json_data, 'beicai_score', 'beicai_event', '被裁员用户风险')
        
        actions = [Actions1, Actions2, Actions3, Actions4, Actions5]
        ACTIONS = [j for i in actions for j in i]
        return ACTIONS

In [67]:
upload = Upload(change.json_data)
ACTIONS = upload.upload()

In [70]:
#批量更新也可以采用如下的方式进行json拼装，最后写入
success, _ = helpers.bulk(es, ACTIONS, index='dc_emp_data', raise_on_error=True)

/home/weijunfei/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


### 批量处理